<a href="https://colab.research.google.com/github/ruthbrennankk/data_visualisation/blob/master/Minard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install altair

In [3]:
#importing libraries
import pandas as pd
import altair as alt
import numpy as np
import math

In [5]:
#reading minard datasheet
minard = pd.read_excel('/content/drive/MyDrive/Year V/Data Visualization/minard-data.xlsx')
minard.head(5)

,LONC,LATC,CITY,LONT,TEMP,DAYS,MON,DAY,LONP,LATP,SURV,DIR,DIV
0,24.0,55.0,Kowno,37.6,0.0,6.0,Oct,18.0,24.0,54.9,340000,A,1
1,25.3,54.7,Wilna,36.0,0.0,6.0,Oct,24.0,24.5,55.0,340000,A,1
2,26.4,54.4,Smorgoni,33.2,-9.0,16.0,Nov,9.0,25.5,54.5,340000,A,1
3,26.8,54.3,Molodexno,32.0,-21.0,5.0,Nov,14.0,26.0,54.7,320000,A,1
4,27.7,55.2,Gloubokoe,29.2,-11.0,10.0,NaN,NaN,27.0,54.8,300000,A,1


In [6]:
#splitting cities from minard data
cities = minard[['LONC','LATC','CITY']]
cities = cities[:20]
cities.head(5)

,LONC,LATC,CITY
0,24.0,55.0,Kowno
1,25.3,54.7,Wilna
2,26.4,54.4,Smorgoni
3,26.8,54.3,Molodexno
4,27.7,55.2,Gloubokoe


In [24]:
#splitting temperature dataset from minard data
temp = minard[['LONT','TEMP','DAYS','MON','DAY']]
temp = temp[:9]

# Make day into a natural number
temp['DAY'] = temp['DAY'].fillna(0).astype(int).map(str)
temp['MON'] = temp['MON'].fillna("0").map(str)

temp['DATE'] = temp['DAY'].map(str) + ' / ' + temp['MON']
temp['DATE'] = temp['DATE'].replace("0 / 0", "N/A")
temp = temp.drop(['MON','DAY'], axis = 1)


temp["label"] = temp.fillna("").apply(
    axis=1, func=lambda row: "{}° - {}".format(row[1], row[3].replace("-", " "))
)
temp.head(5)

,LONT,TEMP,DAYS,DATE,label
0,37.6,0.0,6.0,18 / Oct,0.0° - 18 / Oct
1,36.0,0.0,6.0,24 / Oct,0.0° - 24 / Oct
2,33.2,-9.0,16.0,9 / Nov,-9.0° - 9 / Nov
3,32.0,-21.0,5.0,14 / Nov,-21.0° - 14 / Nov
4,29.2,-11.0,10.0,N/A,-11.0° - N/A


In [10]:
#splitting troops data from minard data sheet
army = minard[['LONP','LATP','SURV','DIR','DIV']]

#connecting the trail between A and R for Divisions 1, 2 and 3
army = army.append(pd.Series([37.6,55.8,100000,'A',1], index=army.columns ), ignore_index=True)
army = army.append(pd.Series([28.7,55.5,33000,'A',2], index=army.columns ), ignore_index=True)
army = army.append(pd.Series([24.6,55.8,22000,'A',3], index=army.columns ), ignore_index=True)

#Divide Up
div_1_a = army[army["DIV"] == 1]
div_1_r = div_1_a[div_1_a["DIR"]=='R']
div_1_a = div_1_a[div_1_a["DIR"]=='A']

div_2_a = army[army["DIV"] == 2]
div_2_r = div_2_a[div_2_a["DIR"]=='R']
div_2_a = div_2_a[div_2_a["DIR"]=='A']

div_3_a = army[army["DIV"] == 3]
div_3_r = div_3_a[div_3_a["DIR"]=='R']
div_3_a = div_3_a[div_3_a["DIR"]=='A']

army.head(5)

,LONP,LATP,SURV,DIR,DIV
0,24.0,54.9,340000,A,1
1,24.5,55.0,340000,A,1
2,25.5,54.5,340000,A,1
3,26.0,54.7,320000,A,1
4,27.0,54.8,300000,A,1


In [1]:
def plot_trail(data, colour) :
  return  alt.Chart(data, title='A VISUALISATION OF TROOP LOSSES DURING NAPOLEON’S RUSSIA CAMPAIGN').mark_trail().encode(
            longitude='LONP',
            latitude='LATP',
            size=alt.Size('SURV', scale=alt.Scale(range=[1, 60]), legend=None),
            detail='DIV',
            color=colour,
            tooltip=[
                alt.Tooltip('DIV', title='Division'),
                alt.Tooltip('SURV', title='Survivors'),
                alt.Tooltip('DIR', title='Direction'),
                alt.Tooltip('LONP', title='Longitude'),
                alt.Tooltip('LATP', title='Latitude')
            ],
            
        ).properties(width=1300,height=350)

In [11]:
# Div One
div1_adv_chart = plot_trail(div_1_a, alt.value('#4287f5'))
div1_ret_chart = plot_trail(div_1_r, alt.value("#9ec3ff"))
div1_chart = div1_adv_chart + div1_ret_chart
# div1_chart

In [12]:
# Div Two
div2_adv_chart = plot_trail(div_2_a, alt.value('#ff9438'))
div2_ret_chart = plot_trail(div_2_r, alt.value("#ffcb9e"))
div2_chart = div2_adv_chart + div2_ret_chart
# div2_chart

In [13]:
# Div Three
div3_adv_chart = plot_trail(div_3_a, alt.value('#0f9432'))
div3_ret_chart = plot_trail(div_3_r, alt.value("#92fcaf"))
div3_chart = div3_adv_chart + div3_ret_chart
# div3_chart

In [14]:
army_chart = div1_chart + div2_chart + div3_chart
army_chart

alt.LayerChart(...)

In [16]:
cities_circle = alt.Chart(cities).mark_circle(
    size=int(50),
    color='black'
).encode(
    # x='LONC:Q',
    longitude="LONC:Q",
    latitude="LATC:Q",
    tooltip=[
        alt.Tooltip('CITY', title='City'),
        alt.Tooltip('LONC', title='Longitude'),
        alt.Tooltip('LATC', title='Latitude')
    ]
).properties(width=1300,height=350)

x_encode = alt.X('LONC:Q',
        scale=alt.Scale(domain=[army["LONP"].min(), army["LONP"].max()]),
        # axis=alt.Axis(title=None, labels=True)
    )

cities_label = alt.Chart(cities).mark_text(
    fontSize=int(10),
    dx=int(0),
    dy=int(12),
    fontStyle='italic'
).encode(
    # x=x_encode,
    longitude="LONC:Q",
    latitude="LATC:Q",
    text="CITY"
).properties(width=1300,height=350)

cities_chart = cities_label + cities_circle
cities_chart

alt.LayerChart(...)

In [17]:
#combining the map charts 
map_chart = army_chart + cities_chart
map_chart = map_chart.properties(width = 900, height = 600)
map_chart

alt.LayerChart(...)

In [25]:
temp_chart = alt.Chart(temp, title='A VISUALISATION OF TEMPERATURE DURING RETREAT OF NAPOLEON’S RUSSIA CAMPAIGN IN DEGREES').mark_trail().encode(x='LONT',y='TEMP',)
temp_chart = temp_chart.properties(width=1300,height=350)
temp_chart

alt.Chart(...)

In [ ]:
temp_circle_chart = alt.Chart(temp).mark_circle(
    size=int(50),
    color='black'
).encode(
    x='LONT',
    y='TEMP',
    tooltip=[
        alt.Tooltip('LONT', title='Longitude'),
        alt.Tooltip('TEMP', title='Temperature'),
    ]
).properties(width=1300,height=350)

temp_text_chart = alt.Chart(temp).mark_text(
    fontSize=int(10),
    dx=int(50),
    dy=int(-6),
    fontStyle='italic'
).encode(
    x='LONT',
    y='TEMP',
    text='label'
).properties(width=1300,height=350)

temp_label_chart = temp_circle_chart + temp_text_chart
temp_label_chart

In [27]:
temp = temp_chart + temp_label_chart
temp

alt.LayerChart(...)

In [28]:
#combining temp and map chart
final_chart = alt.vconcat(map_chart, temp).configure_view(
    width=800,
    height=200,
    strokeWidth=0
).configure_axis(
    grid=False,
    labelFont="Calibri",
    titleFont="Calibri"
)
final_chart.background='white'
final_chart

alt.VConcatChart(...)